I came across an interesting way to setup mock objects the other day in python. I was trying to mock out a call to the database. I always have difficulties mocking context managers, because I so often forget that it is the ``return_value`` of ``__enter__`` that needs to be patched. Anyway the code that I was trying to mock looked something like this.

In [1]:
import pymssql

def load_data(server, database):
    with pymssql.connect(server, database, user='', password='', as_dict=True) as conn:
        with conn.cursor() as cursor:
            cursor.callproc('pLoadData')
            result = cursor.fetchone()
            # do some stuff with result
            return result

After some head scratching and several attempts I managed to get the mock setup to patch in my own result to ``fetchone``.

In [2]:
import unittest.mock

with unittest.mock.patch('pymssql.connect') as connect:
    
    connect.return_value.__enter__.return_value \
        .cursor.return_value.__enter__.return_value \
            .fetchone.return_value = 42
            
    print(load_data('server', 'database'))

42


However this is really cumbersome, error prone and easy to forget. However because of the way ``Mock`` and ``MagicMock`` work (by creating accessed attributes as new child mocks if the attribute doesn't exist) we can just invoke the functions in ``with`` blocks and the setup code is automatically generated. When the function under test is invoked the attributes will already exist, and the mock objects I already set up are used by the code under test. This is much more readable than the chaining of multiple ``return_value`` and ``__enter__``.

In [3]:
with unittest.mock.patch('pymssql.connect') as connect:
    
    with connect() as conn:
        with conn.cursor() as cursor:
            cursor.fetchone.return_value = 42
            
    print(load_data('server', 'database'))

42
